In [1]:
import os
import csv
import pylab
import numpy
import pandas as pd
from itertools import zip_longest

# Defasc: make maxpairwise_pop.csv
## Dendrite lengths normalized to head size

In [2]:
def maxpairwise_pop(genotype, timepoint):
    """
    input: genotype, timepoint (strings)
    output: maxpairwise_pop.csv and middle_dendr_pop.csv
    returns: file_name
    """
    # open files
    script_dir = os.path.dirname('__file__') #<-- absolute dir the script is in
    rel_path = 'SADA csv files binned/' + genotype + '/'
    abs_file_path_pre = os.path.join(script_dir, rel_path)
    prefix = [folder for folder in os.listdir(abs_file_path_pre) if folder.startswith(timepoint)]
    abs_file_path = os.path.join(abs_file_path_pre, prefix[0])
    
    # create maxpairwise_population.csv with appropriate column titles
    header_de = ['animal', 'whole_bundle_max_in_pixels', 
              'whole_bundle_length_in_pixels', 
              'max_values(proximal_to_distal),by_pixel']
    with open('./SADA csv files binned/' + genotype + '/' + genotype + '_' + timepoint + '_maxpairwise_pop.csv', 'w') as first_row_de:
        writer_de = csv.writer(first_row_de)
        writer_de.writerow(header_de)
    first_row_de.close()

    # Make a new csv file to input values
    header = ['animal', 'middle_dendrite']
    with open('./SADA csv files binned/' + genotype + '/' + genotype + '_' + timepoint + '_middle_dendr_pop.csv', 'w') as first_row:
        writer = csv.writer(first_row)
        writer.writerow(header)
    first_row.close()

    # iterate the following processes for every file in the folder
    for file in os.listdir(abs_file_path):
        if file.endswith(".csv"):
            file_name = file
            file_to_open = os.path.join(abs_file_path, file)
            # open the file
            f = open(file_to_open) 
            csv_f = csv.reader(f)
            next(csv_f) #skip the first row of column titles


            # get RG, BR, GB distances
            pairwise_distances = []            
            for row_de in csv_f:
                b_de = [float(cell_de) for cell_de in row_de[3:6]]
                pairwise_distances.append(b_de)

            pairwise_distances = numpy.transpose(pairwise_distances)
            RG_dist = pairwise_distances[0]
            BR_dist = pairwise_distances[1]
            GB_dist = pairwise_distances[2]


            # cut off pixels toward cell body for all pairwise distances
            def normalization(RG_dist, BR_dist, GB_dist):
                normalization_factor_microns = int(file[0:2])
                
                # adjust normalization factor according to image magnification (40x/60x/100x)
                if '100x' in prefix[0].lower():
                    normalization_factor_pixels = 15 * normalization_factor_microns
                elif '40x' in prefix[0].lower():
                    normalization_factor_pixels = 6 * normalization_factor_microns
                elif '60x' in prefix[0].lower() and timepoint == '48h': # 60x
                    normalization_factor_pixels = int(9 * normalization_factor_microns * 0.9) # 90% of the anatomical distance
                elif '60x' in prefix[0].lower() and timepoint == '24h':
                    normalization_factor_pixels = int(9 * normalization_factor_microns * 0.8) # 80% of the anatomical distance                    
                elif '60x' in prefix[0].lower() and timepoint == '72h':
                    normalization_factor_pixels = int(9 * normalization_factor_microns * 1.0) # 100% of the anatomical distance                    
                
                zeros = [0] * (normalization_factor_pixels)
                RG_dist_de = [x_rg + y_rg for x_rg, y_rg in zip_longest(zeros, RG_dist, fillvalue=0)]
                BR_dist_de = [x_br + y_br for x_br, y_br in zip_longest(zeros, BR_dist, fillvalue=0)]
                GB_dist_de = [x_gb + y_gb for x_gb, y_gb in zip_longest(zeros, GB_dist, fillvalue=0)]

                RG_dist_de = RG_dist_de[:normalization_factor_pixels]
                BR_dist_de = BR_dist_de[:normalization_factor_pixels]
                GB_dist_de = GB_dist_de[:normalization_factor_pixels]

                normalized_pairwise = RG_dist_de, BR_dist_de, GB_dist_de
                return normalized_pairwise

            normalized_pairwise_distances = normalization(RG_dist, BR_dist, GB_dist)

            # bin distances to a set number of bins
            def binned_dist(RG_dist, BR_dist, GB_dist, bins=100):
                pixels = len(RG_dist)
                bin_size = pixels/bins
                n=0
                bin_index = []
                while n <= pixels:
                    bin_index.append(round(n))
                    n += bin_size

                RG_binned = []
                BR_binned = []
                GB_binned = []

                n_bins = 0
                while n_bins < (len(bin_index)-1):
                    RG_bin = sum(RG_dist[bin_index[n_bins] : bin_index[n_bins + 1]]) / (-bin_index[n_bins] + bin_index[n_bins + 1])
                    RG_binned.append(RG_bin)
                    BR_bin = sum(BR_dist[bin_index[n_bins] : bin_index[n_bins + 1]]) / (-bin_index[n_bins] + bin_index[n_bins + 1])
                    BR_binned.append(BR_bin)
                    GB_bin = sum(GB_dist[bin_index[n_bins] : bin_index[n_bins + 1]]) / (-bin_index[n_bins] + bin_index[n_bins + 1])
                    GB_binned.append(GB_bin)
                    n_bins += 1

                binned_distances = RG_binned, BR_binned, GB_binned
                binned_distances_transposed = numpy.transpose(binned_distances)
                                
                return binned_distances_transposed

            binned_distances_transposed = binned_dist(normalized_pairwise_distances[0], 
                                                      normalized_pairwise_distances[1], 
                                                      normalized_pairwise_distances[2], 
                                                      100)
            
            # triangles for Max
            #df = pd.DataFrame(binned_distances_transposed, columns =['rg','rb', 'gb'])
            #df.to_csv(file_name + '.csv')
            
            
            def plot_pairwise_dist(binned_distances_transposed):
                binned_dist = numpy.transpose(binned_distances_transposed)

                # plot pairwise distances for each animal
                pylab.figure()
                pylab.plot(binned_dist[0], 'orange')
                pylab.plot(binned_dist[1], 'purple')
                pylab.plot(binned_dist[2], 'yellow')                
                pylab.ylim([0.0, 45.0]) # check this!
                pylab.title(file_name)
                pylab.savefig(file_name + 'pairwise_dist.svg')
                pylab.close()                        
#             plot_pairwise_dist(binned_distances_transposed)
            
    
            # find the maximum value per row of pairwise distances
            max_values_de = []
            for row_de in binned_distances_transposed:
                b_de = [float(cell_de) for cell_de in row_de[:]]
                max_row_de = max(b_de[:])
                max_values_de.append(max_row_de)

            # include animal name and other whole-bundle metrics to max_values  
            whole_bundle_max_de = [max(max_values_de)]
            whole_bundle_length_de = [len(max_values_de)]
            output_data_de = whole_bundle_max_de + whole_bundle_length_de + max_values_de
            output_data_de.insert(0, f.name[:-4])


            # put all of this data into a csv file
            with open('./SADA csv files binned/' + genotype + '/' + genotype + '_' + timepoint + '_maxpairwise_pop.csv', 'a') as new_file_de:
                writer_de = csv.writer(new_file_de)
                writer_de.writerow(output_data_de)

            f.close()
            new_file_de.close()

            # make middle dendrite population.csv
            mydict = {0:0, 1:3, 2:1, 3:2}
            
            
            
            # middle dendrite (opposite longest pairwise distance)
            def middledendrite(binned_distances_transposed):
                """
                looking for middle dendrite points as defined by the point across the longest pairwise distance
                input: binned_distances_transposed
                returns: mid_dendrite_col
                """
                # get middle dendrite column
                mid_dendrite_col = []

                for row in binned_distances_transposed:
                    if row[0] == row[1] == row[2]:
                        mid_dendrite = 0
                    elif row[0] == max(row):
                        mid_dendrite = 1
                    elif row[1] == max(row):
                        mid_dendrite = 2
                    elif row[2] == max(row):
                        mid_dendrite = 3
                    else: 
                        mid_dendrite = 0

                    # convert WT values that start with 072 to the right color order
                    if file_name[3:6] == '072':
                        mid_dendrite = mydict[mid_dendrite]                    
                    mid_dendrite_col.append(mid_dendrite)
                return mid_dendrite_col

            
            
            
            
            def farthest(binned_distances_transposed):
                """
                looking for farthest dendrite point as defined by the point across the shortest pairwise distance
                input: binned_distances_transposed
                returns: farthest_dendrite_col
                """
                # get farthest dendrite column
                farthest_dendrite_col = []

                for row in binned_distances_transposed:
                    if row[0] == row[1] == row[2]:
                        far_dendrite = 0
                    elif row[0] == min(row):
                        far_dendrite = 1
                    elif row[1] == min(row):
                        far_dendrite = 2
                    elif row[2] == min(row):
                        far_dendrite = 3
                    else: 
                        far_dendrite = 0

                    # convert WT values that start with 072 to the right color order
                    if file_name[3:6] == '072':
                        far_dendrite = mydict[far_dendrite]                    
                    farthest_dendrite_col.append(far_dendrite)
                return farthest_dendrite_col
            

            
            def oddoneout(binned_distances_transposed):
                """
                looking for dendrite point that is neither the nearest nor the farthest dendrite point
                input: binned_distances_transposed
                returns: odd_dendrite_col
                """
                # get odd dendrite column
                odd_dendrite_col = []

                for row in binned_distances_transposed:
                    if row[0] == row[1] == row[2]:
                        odd_dendrite = 0
                    elif row[0] not in [max(row), min(row)]:
                        odd_dendrite = 1
                    elif row[1] not in [max(row), min(row)]:
                        odd_dendrite = 2
                    elif row[2] not in [max(row), min(row)]:
                        odd_dendrite = 3
                    else:
                        odd_dendrite = 0

                    # convert WT values that start with 072 to the right color order
                    if file_name[3:6] == '072':
                        odd_dendrite = mydict[odd_dendrite]                    
                    odd_dendrite_col.append(odd_dendrite)
                return odd_dendrite_col            
            
            

            
            # to get farthest dendrite heatmap and summary plot
#             mid_dendrite_col = farthest(binned_distances_transposed)

            # to get middle dendrite heatmap and summary plot
            mid_dendrite_col = middledendrite(binned_distances_transposed)

            # to get odd dendrite heatmap and summary plot
#             mid_dendrite_col = oddoneout(binned_distances_transposed)

            
            # get animal name
            mid_dendrite_col.insert(0,f.name[:-4])



            # put all of this data into a csv file
            with open('./SADA csv files binned/' + genotype + '/' + genotype + '_' + timepoint + '_middle_dendr_pop.csv', 'a') as new_file:
                writer = csv.writer(new_file)
                writer.writerow(mid_dendrite_col)

            new_file.close()
    return file_name

In [3]:
def oddoneout(binned_distances_transposed):
    """
    looking for dendrite point that is neither the nearest nor the farthest dendrite point
    input: binned_distances_transposed
    returns: odd_dendrite_col
    """
    # get odd dendrite column
    odd_dendrite_col = []

    for row in binned_distances_transposed:
        if row[0] == row[1] == row[2]:
            odd_dendrite = 0
        elif row[0] not in [max(row), min(row)]:
            odd_dendrite = 1
        elif row[1] not in [max(row), min(row)]:
            odd_dendrite = 2
        elif row[2] not in [max(row), min(row)]:
            odd_dendrite = 3
        else:
            odd_dendrite = 0
        
        # convert WT values that start with 072 to the right color order
        if file_name[3:6] == '072':
            odd_dendrite = mydict[odd_dendrite]                    
        odd_dendrite_col.append(odd_dendrite)
    return odd_dendrite_col

# Make maxpairwise distance plot

In [4]:
def maxpairwiseplot(genotype, timepoint, smooth_window=5):
    """
    genotype: string, genotype
    timepoint: string, timepoint
    smooth_window: int, smoothing window
    returns: None
    """
    # open maxpairwise plot
    maxpairwise = open('./SADA csv files binned' + '/' + genotype + '/' + genotype + '_' + timepoint + '_maxpairwise_pop.csv', 'r')
    csv_mpw = csv.reader(maxpairwise)
    next(csv_mpw)
    
    pylab.figure()

    # plot smoothed max pairwise distances
    for each_row in csv_mpw:
        row_ints = [float(i) for i in each_row[3:]]
        smoothed_values = movingaverage(row_ints, smooth_window)
        pylab.plot(smoothed_values, 'blue')
    
    # plot niceties
    if timepoint == '48h' or timepoint == '24h':
        pylab.ylim([0.0, float(9*5)]) # 5 micron y-range at 60x objective
        pylab.yticks(numpy.arange(0, float(9*5)+1, 9))

    elif timepoint == '72h' and genotype == 'WT':
        pylab.ylim([0.0, float(6*5)]) # 5 micron y-range at 40x objective
        pylab.yticks(numpy.arange(0, float(6*5)+1, 6))

    elif timepoint == '72h':
        pylab.ylim([0.0, float(9*5)]) # 5 micron y-range at 60x objective
        pylab.yticks(numpy.arange(0, float(9*5)+1, 9))

    else:
        print ('warning, y axis numbers for maxpairwise dist plot incorrect')
    pylab.xlabel('distance along dendrite, 100 bins')
    pylab.ylabel('maximum pairwise distance, 5 microns')
    pylab.title('maxpairwise distance plot')
    
    pylab.savefig(maxpairwise.name[0:-7] + '_plot.svg')
    maxpairwise.close()
    pylab.close()

# Make middle dendrite population heatmap

In [5]:
def heatmap(genotype, timepoint, f_name):
    """
    function: plots all values by color (make a heatmap)
    inputs:
    genotype = string, genotype
    timepoint = string, timepoint
    f_name = string; name of csv file for individual animals (f.name)
    returns: value_points_array, g_name (middle_dendrite_pop.csv name)
    """
    # open middle_dendr_pop.csv
    g = open('./SADA csv files binned/' + genotype + '/' + genotype + '_' + timepoint + '_middle_dendr_pop.csv', 'r')
    csv_g = csv.reader(g)
    next(csv_g)

    # get x, y, and intensity values
    animal_names = []
    all_points = []
    value_points = []

    for each_row in csv_g:
        # x-axis labels
        animal = each_row[0]
        animal_names.append(animal[:-11])

        # y-axis minimum
        number_of_points = len(each_row[1:])
        all_points.append(number_of_points)
        all_points_min = min(all_points)

        # x values
        value_of_points = list(map(float, each_row[1:]))
        value_points.append(value_of_points)

    value_points_a = []
    for each_row_a in value_points:
        value_of_points_a = each_row_a[1:int(all_points_min)]
        value_points_a.append(value_of_points_a)

    # plot heatmap, colorbar:
    value_points_array = numpy.array(value_points_a)
    from matplotlib.colors import LinearSegmentedColormap
    vmax = 3.0    
    cmap = LinearSegmentedColormap.from_list('mycmap', [(0 / vmax, 'white'),
                                                    (1 / vmax, 'yellow'),
                                                    (2 / vmax, 'red'),
                                                    (3 / vmax, 'blue')])

    fig, ax = pylab.subplots()
    heatmap = ax.pcolor(value_points_array, cmap=cmap, vmin = 0, vmax = vmax)

    # put the major ticks at the middle of each cell
    ax.set_yticks(numpy.arange(value_points_array.shape[0]) + 0.5, minor=False)
    ax.invert_yaxis()

    # heatmap labels and delete annoying tickmarks
    ax.set_yticklabels(animal_names, minor = False)
    #     magnification = input('Are images taken at (a) 60X or (b) 100X? [a/b]: ')
    #     if magnification == 'a':
    #         x_scaled_ticks = numpy.arange(0,len(value_of_points_a)+1, 92.6)
    #     elif magnification == 'b':
    #         x_scaled_ticks = numpy.arange(0,len(value_of_points_a)+1, 154.31)
    #     else:
    #         print("that's not a valid option.")
    #         import sys
    #         sys.exit()
    #     ax.set_xticks(x_scaled_ticks, minor=False)
    #     ax.set_xticklabels([])
    #     ax.set_xlabel('distance from nose')
    #     ax.get_xaxis().set_tick_params(direction='out')
    ax.tick_params(right='off')
    ax.tick_params(left='off')
    ax.tick_params(top='off')

    # figure title
    pylab.title('n = %s' % len(animal_names))

    # save the figure
    pylab.savefig(g.name[0:-4] + '_heatmap.svg')
    pylab.clf()
    pylab.cla()
    pylab.close()
    g.close()
    
    return value_points_array, g.name

# Make quantitative measure of misfasciculation (summary plot)

In [6]:
def summaryplot(value_points_array, middle_dendrite_pop_name, f_name, smooth_window=5):
    """
    inputs: 
    value_points_array = nested list; all the datapoints for dendrite in the middle acros the popluation
    smooth_window = int; default=5, smoothing window
    middle_dendrite_pop_name = string; middle_dendrite_pop file (g.name)
    f_name = string; name of csv file for individual animals (f.name)
    returns: count_ones, count_twos, count_threes, total_count, smooth_window for chi-squared test
    """
    # transpose value_points_array
    value_points_array_transposed = value_points_array.T

    # get the count of '1', '2', and '3'
    count_ones = []
    count_twos = []
    count_threes = []
    total_count = []
    for transposed_row in value_points_array_transposed:
        count_one = list(transposed_row).count(1.)
        count_ones.append(count_one)
        count_two = list(transposed_row).count(2.)
        count_twos.append(count_two)
        count_three = list(transposed_row).count(3.)
        count_threes.append(count_three)

        # get the total count by sum
        total = count_one + count_two + count_three
        total_count.append(total)

    # get the fraction of 1, 2, 3
    frac_ones = numpy.divide(count_ones,total_count)
    frac_twos = numpy.divide(count_twos,total_count)
    frac_threes = numpy.divide(count_threes,total_count)

    # smooth lines for fraction of 1, 2, 3
    smoothed_frac_ones = movingaverage(frac_ones, int(smooth_window))
    smoothed_frac_twos = movingaverage(frac_twos, int(smooth_window))
    smoothed_frac_threes = movingaverage(frac_threes, int(smooth_window))

    # plot the three lines
    pylab.figure()
    pylab.plot(smoothed_frac_ones, 'yellow')
    pylab.plot(smoothed_frac_twos, 'red')
    pylab.plot(smoothed_frac_threes, 'blue')

        
#     # magnification
#     if magnification == 'a':
#         pylab.xticks(numpy.arange(0, len(value_of_points_a)+1, 92.6))
#     elif magnification == 'b':
#         pylab.xticks(numpy.arange(0, len(value_of_points_a)+1, 154.31))
#     else:
#         print("that's not a valid option.")
#         import sys.exit
#         sys.exit()

    # plot niceties
    pylab.ylim([0.0,1.0])
    pylab.xlabel('distance from nose', fontsize=10)  
    pylab.title('n = %s' % total_count[0])
    pylab.suptitle('quantitative summary plot')
    pylab.tick_params(labelsize=10)
    pylab.tick_params(bottom='off')
    pylab.tick_params(right='off')

    # save the figure
    pylab.savefig(middle_dendrite_pop_name[0:-4] + '_quant.svg')
    pylab.close()
    
    return count_ones, count_twos, count_threes, total_count, smooth_window

# Chi-squared test for misfasciculation (compared to random)

In [7]:
# chi-squared test with three variables
def chisquaredtest(experimental_1, experimental_2, experimental_3, total, middle_dendrite_pop_name, smooth_window):
    """
    input: lists; experimental_1, experimental_2, experimental_3 are counts of 1's, 2's, and 3's respectively.
    total = list; total count
    middle_dendrite_pop_name = string; for file naming
    returns: None
    """
    from scipy.stats import chisquare
    
    # expected value for chi-square
    expected = [value / 3 for value in total]
    
    # get experimental values for chi-squares
    experimental_values = [experimental_1, experimental_2, experimental_3, expected, total]
    experimental_values = numpy.transpose(experimental_values)
    
    # set up empty variables
    chi_squared_values = []
    p_values = []

    # get chi-square and p-values for each bin 
    for experimental_value in experimental_values:
        chi_square = chisquare(experimental_value[0:3], experimental_value[3])
        chi_squared_values.append(chi_square[0])
        p_values.append(chi_square[1])
    
    # smooth values
    smoothed_pvalues = movingaverage(p_values, int(smooth_window))
    x_pvalues = len(smoothed_pvalues)
    
    # create figure of smoothed chi-square p-values
    pylab.figure()
    pylab.plot(smoothed_pvalues, 'blue')
    pylab.plot([0, x_pvalues], [0.05, 0.05], 'gray')

    pylab.savefig(middle_dendrite_pop_name[0:-4] + '_pvalueplot.svg')
    pylab.close()

    # save chi-squared values and p-values to a list if you want
    chi_df = pd.DataFrame()
    chi_df['p_vals'] = smoothed_pvalues
    chi_df.to_csv(middle_dendrite_pop_name[0:-4] + '_chisq.csv')

# Implement this script

In [8]:
# simple moving average function to smooth data
# def movingaverage(values, window):
#     """
#     input: values to smooth (list); window (int)
#     returns: smoothed values
#     """
#     weights = numpy.repeat(1.0, window)/window
#     smas = numpy.convolve(values, weights, 'valid')
#     return smas


def movingaverage(values, window):
    """
    Compute the moving average of y with specified window length.
    Args:
        y: an 1-d pylab array with length N, representing the y-coordinates of the N sample points
        window_length: an integer indicating the window length for computing moving average
    Returns:
        a list with the same length as y storing moving average of y-coordinates of the N sample points
    """
    # create list of smoothed data points
    averages = []
    
    for i in range(len(values)):
        
        # create list of data contained in window
        last_y_slice = i + window + 1
        if last_y_slice >= len(values) + 1:
            last_y_slice = len(values) + 1
        
        sma = pylab.array(values[i:last_y_slice])
        
        # get average of data in window
        average = sma.mean()
        averages.append(average)
    
    # return array of moving averages
    return averages



def implement_demisfasc(genotype, timepoint):
    """
    input: 
        strings; genotype and timepoint (0h, 24h, 48h, 72h)
    returns: 
        None
    outputs:
        CSV files (2): maxpairwise_pop, middle_dendrite_pop. 1, 2, and 3 are corrected.
        figures (3 x .svg): heatmap of middle dendrite points, summary plot, chi-squared values
    """
    smooth_window = 5
    
    # make csv files
    f_name = maxpairwise_pop(genotype, timepoint)
    
    # plot maximum pairwise distances
    maxpairwiseplot(genotype, timepoint, smooth_window)
    
    # make heatmap
    value_points_array, middle_dendrite_pop_name = heatmap(genotype, timepoint, f_name)
    
    # make summary plot
    count_ones, count_twos, count_threes, total_count, smoothwindow = summaryplot(value_points_array, middle_dendrite_pop_name, f_name, smooth_window)
    
    # chi-squared test
    chisquaredtest(count_ones, count_twos, count_threes, total_count, middle_dendrite_pop_name, smoothwindow)
    
    print(genotype, timepoint, 'all done!')


# Execution

In [9]:
implement_demisfasc('WT', '24h')
implement_demisfasc('WT', '48h')
implement_demisfasc('WT', '72h')

WT 24h all done!
WT 48h all done!
WT 72h all done!


In [10]:
%whos

Variable              Type        Data/Info
-------------------------------------------
chisquaredtest        function    <function chisquaredtest at 0x11f27fe18>
csv                   module      <module 'csv' from '//ana<...>da/lib/python3.5/csv.py'>
heatmap               function    <function heatmap at 0x11f27f1e0>
implement_demisfasc   function    <function implement_demisfasc at 0x11f27f730>
maxpairwise_pop       function    <function maxpairwise_pop at 0x11f2769d8>
maxpairwiseplot       function    <function maxpairwiseplot at 0x11f27f158>
movingaverage         function    <function movingaverage at 0x11f27f840>
numpy                 module      <module 'numpy' from '//a<...>kages/numpy/__init__.py'>
oddoneout             function    <function oddoneout at 0x11f27fa60>
os                    module      <module 'os' from '//anac<...>nda/lib/python3.5/os.py'>
pd                    module      <module 'pandas' from '//<...>ages/pandas/__init__.py'>
pylab                 module     